## ( 유튜브 키워드 검색 )
1. 화면에서 키워드를 입력 받는다 (파이썬 데이터분석)
2. selenium 활용 검색창에 입력 후 검색 실행
3. page_source를 받아서 BeautifulSoup 적용
    - 태그  찾기
--------------------------------------------------------------------
4. 데이터 추출 후 DataFrame으로 만들기
    - 개별영상타입 ; 채널명, 영상제목, 영상url, 조회수, 영상등록날짜
    - 재생목록타입 ; 채널명, 재생목록제목, 재생목록url
--------------------------------------------------------------------
5. 함수로 만들어서 처리
    - 개별영상타입 함수 video_item()
    - 재생목록타입 함수 video_playlist()

In [ ]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [ ]:
driver = Chrome('C:/soryeong/chromedriver')

keyword = '파이썬 데이터분석'
url = 'https://www.youtube.com/results?search_query={}'.format(keyword)

driver.get(url)

time.sleep(2)

In [ ]:
'''
elem = driver.find_element_by_name('search_query')
elem.clear()

elem.send_keys('파이썬 데이터분석')
elem.send_keys(Keys.RETURN) '''

In [ ]:
body = driver.find_element_by_tag_name('body') #페이지 정보가 담겨있는 body태그 찾기
body.send_keys(Keys.PAGE_DOWN) #키보드의 page_down을 통해서 스크롤 내리기

In [ ]:
for page_down in range(1,6): #5번 스크롤 내리기
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
#soup

In [ ]:
video_name = soup.select('a#video-title')
video_url= soup.select('a#video-title')
#video_date = soup.select('div#metadata-line > span:nth-child(2)')
#video_views = soup.select('div#metadata-line > span:nth-child(1)')

In [ ]:
name_list = []
channel_list = []
url_list = []
date_list = []
views_list = []

In [ ]:
try:
    for i in range(0, len(video_name)): #len 통일
        
        name_list.append(video_name[i].text.strip()) #영상제목

        channel_list.append(
            soup.find_all('ytd-video-renderer', {'class':'style-scope ytd-item-section-renderer'})[i].find('a',{'class':'yt-simple-endpoint style-scope yt-formatted-string'}).text) #채널명

        date_list.append(soup.find_all('ytd-video-renderer', {'class':'style-scope ytd-item-section-renderer'})[i].find_all('span', {'class':'style-scope ytd-video-meta-block'})[1].text) #등록날짜

        views_list.append(soup.find_all('ytd-video-renderer', {'class':'style-scope ytd-item-section-renderer'})[i].find_all('span', {'class':'style-scope ytd-video-meta-block'})[0].text) #조회수

        url_list.append('{}{}'.format('https://www.youtube.com',video_url[i]['href'])) #url
        
except IndexError:
    pass

In [ ]:
#nth-child 부모 엘리먼트의 모든 자식 엘리먼트중 n번째
#soup.select('div#metadata-line > span:nth-child(2)')[0].text #등록날짜

In [ ]:
#soup.select('div#metadata-line > span:nth-child(1)') #조회수

In [ ]:
#soup.select('a#video-title')[0].get('href')

In [ ]:
item_dict = {'url': url_list,
              '영상제목': name_list,
              '채널명': channel_list,
              '영상등록날짜': date_list,
              '조회수': views_list}

item_dict

item_df = pd.DataFrame.from_dict(item_dict, orient='index')
item_df.T

In [ ]:
#조회수
#soup.find_all('ytd-video-renderer', {'class':'style-scope ytd-item-section-renderer'})[1].find_all('span', {'class':'style-scope ytd-video-meta-block'})[0].text

In [ ]:
#등록날짜
#soup.find_all('ytd-video-renderer', {'class':'style-scope ytd-item-section-renderer'})[0].find_all('span', {'class':'style-scope ytd-video-meta-block'})[1].text

#함수만들기

In [1]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

def video_item(keyword):

    driver = Chrome('C:/soryeong/chromedriver')

    keyword = keyword
    url = 'https://www.youtube.com/results?search_query={}'.format(keyword)

    print('Youtube 접속 및 검색중')
    
    driver.get(url)
    
    time.sleep(2)
    
    print('스크롤 내리는중')

    body = driver.find_element_by_tag_name('body') #페이지 정보가 담겨있는 body태그 찾기
    body.send_keys(Keys.PAGE_DOWN) #키보드의 page_down을 통해서 스크롤 내리기

    for page_down in range(1,6): #5번 스크롤 내리기
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    video_name = soup.select('a#video-title')
    video_url= soup.select('a#video-title')

    #개별영상 list 생성
    name_list = []
    channel_list = []
    url_list = []
    date_list = []
    views_list = []

    try:
        for i in range(0, len(video_name)): #len 통일

            name_list.append(video_name[i].text.strip()) #영상제목

            channel_list.append(
                soup.find_all('ytd-video-renderer', {'class':'style-scope ytd-item-section-renderer'})[i].find('a',{'class':'yt-simple-endpoint style-scope yt-formatted-string'}).text) #채널명

            date_list.append(soup.find_all('ytd-video-renderer', {'class':'style-scope ytd-item-section-renderer'})[i].find_all('span', {'class':'style-scope ytd-video-meta-block'})[1].text) #등록날짜

            views_list.append(soup.find_all('ytd-video-renderer', {'class':'style-scope ytd-item-section-renderer'})[i].find_all('span', {'class':'style-scope ytd-video-meta-block'})[0].text) #조회수

            url_list.append('{}{}'.format('https://www.youtube.com',video_url[i]['href'])) #url

    except IndexError:
        pass
    
    item_dict = {'url': url_list,
              '영상제목': name_list,
              '채널명': channel_list,
              '영상등록날짜': date_list,
              '조회수': views_list}
    #youtube_dict
    
    item_df = pd.DataFrame.from_dict(item_dict, orient='index')
    
    return item_df.T

In [2]:
video_item('파이썬 데이터분석')

Youtube 접속 및 검색중
스크롤 내리는중


,url,영상제목,채널명,영상등록날짜,조회수
0,https://www.youtube.com/watch?v=8_VWnRvbGPs,"파이썬 데이터 분석 - 설치부터 라이브러리 예제까지 (feat. 넘파이, 맷플롯립,...",GongbroDesk,7개월 전,조회수 2.9만회
1,https://www.youtube.com/watch?v=lspu830SzC8,[1/29] 파이썬 데이터 분석을 위한 Pandas 기초 - 판다스 10분 완성소개...,todaycode오늘코드,2년 전,조회수 3.2만회
2,https://www.youtube.com/watch?v=3-iwY0_u0VA,파이썬 판다스로 회사 엑셀데이터 전처리 하는 방법 ! 1탄 (Python / Pan...,데이터 스테이션,5개월 전,조회수 5.8천회
3,https://www.youtube.com/watch?v=uJ6QO6jLBWA,"[1/5] 파이썬 데이터 분석 - 전국 도시 공원 표준 데이터 불러오기, 요약하기",todaycode오늘코드,2년 전,조회수 1.3만회
4,https://www.youtube.com/watch?v=Dfpi4pAzn78,"[통계, 파이썬] 데이터분석 능력 기르기 1. 이건 알고 시작하자.",Data Scientist이지영,1년 전,조회수 8.5천회
5,https://www.youtube.com/watch?v=JoL3ACNDs-I,"25분만에 끝내는 데이터분석, 머신러닝을 위한 기본 파이썬",테디노트,1년 전,조회수 1만회
6,https://www.youtube.com/watch?v=1e2IjkDYE_8,무슨 분석 툴을 배워야 할까? - 실제 데이터 과학/데이터 분석 업계에서의 R 과 ...,Visualising Korea,1년 전,조회수 2.5만회
7,https://www.youtube.com/watch?v=19oPYr6d5IY,[파이썬] 데이터 분석을 위한 파이썬 [08] Pandas (Python for D...,메모밍 채널 ( Channel Memoming ),2년 전,조회수 4.8천회
8,https://www.youtube.com/watch?v=Rh_gJz26BWw,12-1. 파이썬 데이터 분석 소개 및 csv 파일 데이터 읽어오기 (What is...,파이썬멘토 조인석,1년 전,조회수 722회
9,https://www.youtube.com/watch?v=yQ20jZwDjTE,파이썬 코딩 무료 강의 (활용편3) - 웹 크롤링? 웹 스크래핑! 제가 가진 모든 ...,데이터 스테이션,1년 전,조회수 7.5천회


#재생목록

In [ ]:
playlist = soup.find_all('ytd-playlist-renderer', {'class':'style-scope ytd-item-section-renderer'})

In [ ]:
playlist_name = []
playlist_url = []
playlist_channel = []

In [ ]:
try:
    for j in range(0, len(playlist)):
        playlist_name.append(playlist[j].find('span', {'class':'style-scope ytd-playlist-renderer'}).text.strip())

        playlist_url.append('{}{}'.format('https://www.youtube.com', playlist[j].find('a', {'class':'yt-simple-endpoint style-scope ytd-playlist-renderer'})['href']))

        playlist_channel.append(playlist[j].find('a', {'class':'yt-simple-endpoint style-scope yt-formatted-string'}).text)

except IndexError:
    pass

In [ ]:
#재생목록제목
soup.find_all('ytd-playlist-renderer', {'class':'style-scope ytd-item-section-renderer'})[0].find('span', {'class':'style-scope ytd-playlist-renderer'}).text.strip()

In [ ]:
#재생목록url
soup.find_all('ytd-playlist-renderer', {'class':'style-scope ytd-item-section-renderer'})[0].find('a', {'class':'yt-simple-endpoint style-scope ytd-playlist-renderer'})['href']

In [ ]:
#채널명
soup.find_all('ytd-playlist-renderer', {'class':'style-scope ytd-item-section-renderer'})[0].find('a', {'class':'yt-simple-endpoint style-scope yt-formatted-string'}).text

In [ ]:
playlist_dict = {'재생목록url': playlist_url,
                '재생목록제목': playlist_name,
                '채널명':playlist_channel}

playlist_dict

playlist_df = pd.DataFrame.from_dict(playlist_dict, orient = 'index')
playlist_df.T

#함수만들기

In [3]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

def video_palylist(keyword):

    driver = Chrome('C:/soryeong/chromedriver')

    keyword = keyword
    url = 'https://www.youtube.com/results?search_query={}'.format(keyword)

    print('Youtube 접속 및 검색중')
    
    driver.get(url)
    
    time.sleep(2)
    
    print('스크롤 내리는중')

    body = driver.find_element_by_tag_name('body') #페이지 정보가 담겨있는 body태그 찾기
    body.send_keys(Keys.PAGE_DOWN) #키보드의 page_down을 통해서 스크롤 내리기

    for page_down in range(1,6): #5번 스크롤 내리기
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    playlist = soup.find_all('ytd-playlist-renderer', {'class':'style-scope ytd-item-section-renderer'})

    #플레이리스트 list 생성
    playlist_name = []
    playlist_url = []
    playlist_channel = []

    try:
        for j in range(0, len(playlist)):
            playlist_name.append(playlist[j].find('span', {'class':'style-scope ytd-playlist-renderer'}).text.strip())

            playlist_url.append('{}{}'.format('https://www.youtube.com', playlist[j].find('a', {'class':'yt-simple-endpoint style-scope ytd-playlist-renderer'})['href']))

            playlist_channel.append(playlist[j].find('a', {'class':'yt-simple-endpoint style-scope yt-formatted-string'}).text)

    except IndexError:
        pass
    
    playlist_dict = {'재생목록url': playlist_url,
                '재생목록제목': playlist_name,
                '채널명':playlist_channel}

    #playlist_dict

    playlist_df = pd.DataFrame.from_dict(playlist_dict, orient = 'index')
    
    return playlist_df.T

In [4]:
video_palylist('파이썬 데이터분석')

Youtube 접속 및 검색중
스크롤 내리는중


,재생목록url,재생목록제목,채널명
0,https://www.youtube.com/watch?v=K0YPltIT7jc&li...,이것이 데이터 분석이다 with 파이썬,한빛미디어
1,https://www.youtube.com/watch?v=XjfUcQk8PiE&li...,[Python] 데이터분석을 위한 파이썬,메모밍 채널 ( Channel Memoming )
2,https://www.youtube.com/watch?v=V8Lpf3WCZ4g&li...,Python 데이터 분석과 이미지 처리,동빈나
3,https://www.youtube.com/watch?v=9-_UWTPRlQI&li...,왕초보들을 위한 데이터분석 강의! 모두의 데이터분석 with 파이썬,리송스쿨
